# chap 17 딥러닝을 이용한 자연어 처리

In [4]:
# 모든 문장을 단어로 토큰화
from tensorflow.keras.preprocessing.text import text_to_word_sequence

text = '해보지 않으면 해낼 수 없다'
print("text : ", text)

print("토큰화 : ", text_to_word_sequence(text))  # 단어를 토큰화 시켜 리스트 반환

text :  해보지 않으면 해낼 수 없다
토큰화 :  ['해보지', '않으면', '해낼', '수', '없다']


In [5]:
# 토크나이저 클래스를 활용하여 단어를 토큰화 한 다음 단어의 개수를 셀 수 있음
from tensorflow.keras.preprocessing.text import Tokenizer

docs = ['먼저 텍스트의 각 단어를 나누어 토큰화합니다.',
       '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
       '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.']

In [ ]:
token = Tokenizer()      # 토큰화 클래스 intance 생성
token.fit_on_texts(docs)  # 토큰화 시킴

In [11]:
print(token.word_counts)   # 토큰화한 각 단어의 출현 빈도 수
print()
print(token.document_count) # 문장의 수
print()
print(token.word_docs)   # 단어가 나오는 문장의 수
print()
print(token.index_word)  # 단어의 인덱스

OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])

3

defaultdict(<class 'int'>, {'먼저': 1, '토큰화합니다': 1, '텍스트의': 2, '각': 1, '단어를': 1, '나누어': 1, '단어로': 1, '인식됩니다': 1, '딥러닝에서': 2, '토큰화해야': 1, '결과는': 1, '있습니다': 1, '토큰화한': 1, '수': 1, '사용할': 1})

{1: '텍스트의', 2: '딥러닝에서', 3: '먼저', 4: '각', 5: '단어를', 6: '나누어', 7: '토큰화합니다', 8: '단어로', 9: '토큰화해야', 10: '인식됩니다', 11: '토큰화한', 12: '결과는', 13: '사용할', 14: '수', 15: '있습니다'}


In [12]:
print('원문 : ',docs)

def word_counts(docs):
    results = {}
    for sentence in docs:
        words = sentence.split(' ')
        for word in words:
            if word in results:
                results[word] += 1
            else:
                results[word] = 1
    return results

results = word_counts(docs)
for word, cnt in results.items():
    print(word, ' : ', cnt)

원문 :  ['먼저 텍스트의 각 단어를 나누어 토큰화합니다.', '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.', '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.']
먼저  :  1
텍스트의  :  2
각  :  1
단어를  :  1
나누어  :  1
토큰화합니다.  :  1
단어로  :  1
토큰화해야  :  1
딥러닝에서  :  2
인식됩니다.  :  1
토큰화한  :  1
결과는  :  1
사용할  :  1
수  :  1
있습니다.  :  1


In [13]:
# 단어의 원-핫 인코딩
# 1. 문장을 토크나이징
# 2. 단어의 인덱스 확인 -> 변수에 추가
# 3. 단어의 인덱스를 원-핫 인코딩 -> 인덱스 갯수 + 1 로 to_categorical

text= '오랫동안 꿈꾸는 이는 그 꿈을 닮아간다.'

token=Tokenizer()
token.fit_on_texts([text])   # 문장을 토큰화 함
# 단어의 인덱스 확인
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [16]:
x_index = token.texts_to_sequences([text])  # 단어의 인덱스만 리스트로 반환

In [22]:
# x_index에 대해 원-핫 인코딩
from tensorflow.keras.utils import to_categorical

to_categorical(x_index, num_classes=len(x_index[0])+1)

array([[[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1.]]], dtype=float32)

In [24]:
## 딥러닝을 활용한 자연어 처리
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical

import numpy as np

In [25]:
# 텍스트 리뷰 자료
docs = ['너무 재밌네요.','최고예요.','참 잘 만든 영화예요.','추천하고 싶은 영화입니다.','한 번 더 보고싶네요.',
       '글쎄요.','별로예요.','생각보다 지루하네요.','연기가 어색해요.','재미없어요.']

# 긍정리뷰는 1, 부정리뷰는 0로 클래스를 지정
classes = np.array([1,1,1,1,1,0,0,0,0,0])

In [26]:
# 자연어 전처리 -> 토큰화
token = Tokenizer()
token.fit_on_texts(docs)   # 단어의 토큰화 시킴
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한': 11, '번': 12, '더': 13, '보고싶네요': 14, '글쎄요': 15, '별로예요': 16, '생각보다': 17, '지루하네요': 18, '연기가': 19, '어색해요': 20, '재미없어요': 21}


In [27]:
# word_index 의 인덱스만 가져옴
x = token.texts_to_sequences(docs)   # 단어를 토큰화 하고 인덱스만 가져옴

# 서로 다른 길이의 데이터를 padding -> 4로
padded_x = pad_sequences(x, 4)
print("패딩한 결과 출력 : ", padded_x)

패딩한 결과 출력 :  [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [11 12 13 14]
 [ 0  0  0 15]
 [ 0  0  0 16]
 [ 0  0 17 18]
 [ 0  0 19 20]
 [ 0  0  0 21]]


In [29]:
# 임베딩에 입력된 단어의 수를 지정
word_size = len(token.word_index) +1

# 단어 임베딩을 포함한 딥러닝 모델 생성
model=Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              176       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


In [30]:
# model 실행 옵션 설정
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)

Epoch 1/20
1/1 [==============================] - 1s 627ms/step - loss: 0.6982 - accuracy: 0.4000
Epoch 2/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6957 - accuracy: 0.4000
Epoch 3/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.4000
Epoch 4/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6906 - accuracy: 0.4000
Epoch 5/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6881 - accuracy: 0.4000
Epoch 6/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6856 - accuracy: 0.6000
Epoch 7/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6831 - accuracy: 0.6000
Epoch 8/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6807 - accuracy: 0.8000
Epoch 9/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6782 - accuracy: 0.8000
Epoch 10/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6758 - accuracy: 0.9000
Epoch 11/20
1/1 [===============

In [32]:
# test 점수 확인
print("Accuracy : {:4f}".format(model.evaluate(padded_x, classes)[1]))

1/1 [==============================] - 0s 31ms/step - loss: 0.6489 - accuracy: 1.0000
Accuracy : 1.000000


In [39]:
model.predict(padded_x[7].reshape(1,-1))

1/1 [==============================] - 0s 27ms/step


array([[0.47410575]], dtype=float32)